In [62]:
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import glob
import pickle

In [57]:
model_path = "D:/Betafinder/Betafinder/models/object_detection/yolo_weights/bounding_and_color.pt"
img_paths = "D:\Betafinder\data\Object_detection_holds"

In [63]:
### Converts the bounding box metric xywh from Yolo output to x1, x2, y1, y2 for chaining
### with cv2. Keeps the predicted color and color name of each hold as well.
def yolobbox2bbox(out, with_color = True):
    all_boxes = []
    plc = out.boxes.xywh
    clr = out.boxes.cls
    clr_names = out.names

    def center_border_to_xxyy(x,y,w,h):
        x1, y1 = x-w//2, y-h//2
        x2, y2 = x+w//2, y+h//2
        return x1, y1, x2, y2

    for i in range(len(plc)):
        to_int = [int(x) for x in plc[i]]
        color = int(clr[i])
        clr_name = clr_names[color]
        if with_color:
            all_boxes.append(((color, clr_name), center_border_to_xxyy(*to_int)))
        else:
            all_boxes.append(center_border_to_xxyy(*to_int))

    return all_boxes

In [66]:
# initialize model and predict
model = YOLO(model_path)

training_set = []

for indx, img in enumerate(glob.glob(img_paths + "/*")):
    if indx % 2 == 0: # Only have labels for imgs 1,3 and 5 
        out = model(img, save = True, save_txt = True)[0]
        out = yolobbox2bbox(out)
        training_set.append(out)


image 1/1 D:\Betafinder\data\Object_detection_holds\IMG20240218163532.jpg: 640x480 1 Blue, 4 Greens, 1 Pink, 2 Purples, 2 Reds, 3 UNKNOWNs, 1 White, 13 Yellows, 82.5ms
Speed: 4.0ms preprocess, 82.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict4
1 label saved to runs\detect\predict4\labels

image 1/1 D:\Betafinder\data\Object_detection_holds\IMG20240218163706.jpg: 640x480 7 Blacks, 1 Gray, 6 Pinks, 3 UNKNOWNs, 79.0ms
Speed: 2.6ms preprocess, 79.0ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict4
2 labels saved to runs\detect\predict4\labels

image 1/1 D:\Betafinder\data\Object_detection_holds\IMG20240218163716.jpg: 640x480 5 Blacks, 5 Blues, 2 Grays, 2 Pinks, 6 Purples, 2 UNKNOWNs, 2 Whites, 96.7ms
Speed: 4.2ms preprocess, 96.7ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict4
3 labels saved to runs\detect\predict4\label

In [67]:
def select_boulder(holds, color):
    boulder = [x for x in holds if x[0][1] == color or x[0][1] == "Gray" or x[0][1] == "UNKNOWN"]
    return boulder

In [85]:
colors = ["Yellow", "Red", "Blue"]

boulders = [select_boulder(training_set[i], colors[i]) for i in range(len(colors))]

boulders = [[y[i][1] for i in range(len(y))] for y in boulders] # Removing color notation

with open("D:\Betafinder\Betafinder/betafinder_dataset/train\input/train1.pkl", "wb") as file:
   pickle.dump(boulders, file)

In [71]:
boulders

[[((10, 'Yellow'), (2370, 2, 2782, 188)),
  ((10, 'Yellow'), (1636, 2251, 2178, 2663)),
  ((10, 'Yellow'), (651, 2363, 855, 2567)),
  ((10, 'Yellow'), (1755, 788, 2087, 1092)),
  ((10, 'Yellow'), (615, 2134, 949, 2324)),
  ((10, 'Yellow'), (922, 3218, 1306, 3444)),
  ((10, 'Yellow'), (1416, 1367, 1754, 1599)),
  ((10, 'Yellow'), (1323, 1590, 1663, 1798)),
  ((10, 'Yellow'), (1133, 2935, 1353, 3081)),
  ((10, 'Yellow'), (1813, 566, 2041, 754)),
  ((10, 'Yellow'), (82, 3382, 196, 3492)),
  ((8, 'UNKNOWN'), (594, 3772, 910, 4018)),
  ((10, 'Yellow'), (905, 3757, 1011, 3849)),
  ((8, 'UNKNOWN'), (451, 596, 1243, 926)),
  ((10, 'Yellow'), (1825, 2776, 2035, 2958)),
  ((8, 'UNKNOWN'), (129, 2987, 405, 3237))],
 [((8, 'UNKNOWN'), (799, 1553, 1331, 1951)),
  ((2, 'Gray'), (1314, 2463, 1776, 2897)),
  ((8, 'UNKNOWN'), (615, 1983, 1067, 2215)),
  ((8, 'UNKNOWN'), (458, 1046, 1286, 1500))],
 [((1, 'Blue'), (1142, 2875, 1536, 3433)),
  ((1, 'Blue'), (1222, 3412, 1718, 3944)),
  ((1, 'Blue'), (989,